In [1]:
import pandas as pd
import copy
import json

In [2]:
df = pd.read_json("data.json")

In [3]:
def get_correct_item(item: dict):
    new_item = copy.deepcopy(item)
    for key, value in item.items():
        if value is None:
            new_item[key] = 'nan'
            continue
        if not isinstance(value, (int, float, str)):
            new_item[key] = str(value)
        if isinstance(value, bool):
            new_item[key] = str(value)
    return new_item


items = []
df.apply(lambda x: items.append(get_correct_item(x.to_dict())), axis=1)

item = items[3]
item

{'id': 'AVpgMuGwLJeJML43KY_c',
 'prices.amountMax': 69.99,
 'prices.amountMin': 69.99,
 'prices.availability': 'Yes',
 'prices.condition': 'New',
 'prices.currency': 'USD',
 'prices.dateSeen': '2017-10-10T05:00:00Z',
 'prices.isSale': 'False',
 'prices.merchant': 'Bestbuy.com',
 'prices.shipping': 'nan',
 'prices.sourceURLs': 'https://www.bestbuy.com/site/boytone-2500w-2-1-ch-home-theater-system-black-diamond/4784804.p%25253FskuId%25253D4784804',
 'asins': 'B018K251JE,B00VILQKQ8',
 'brand': 'Boytone',
 'categories': 'Stereos,Portable Bluetooth Speakers,TV, Video & Home Audio,Speaker Systems,Portable Audio & Video,Electronics,See more Black BOYTONE Bt-210f 30 Watt FM Radio Bluetoo...,Speakers,Home Audio & Theater,All Home Speakers,Consumer Electronics,See more BOYTONE Bt-210f Bluetooth Wireless Speaker Mp3...,Home Theater Systems,MP3 Player Accessories,Home Audio,Audio,Cell,Stereo Shelf Systems',
 'dateAdded': '2015-05-18T14:14:56Z',
 'dateUpdated': '2018-06-13T19:39:02Z',
 'ean': 'nan'

In [4]:
from redis.cluster import RedisCluster as Redis

rc = Redis(host='localhost', port=7000)

In [5]:
rc.get_nodes()

[[host=172.23.0.2,port=7000,name=172.23.0.2:7000,server_type=primary,redis_connection=Redis<ConnectionPool<Connection<host=172.23.0.2,port=7000,db=0>>>],
 [host=172.23.0.5,port=7004,name=172.23.0.5:7004,server_type=replica,redis_connection=Redis<ConnectionPool<Connection<host=172.23.0.5,port=7004,db=0>>>],
 [host=172.23.0.7,port=7001,name=172.23.0.7:7001,server_type=primary,redis_connection=Redis<ConnectionPool<Connection<host=172.23.0.7,port=7001,db=0>>>],
 [host=172.23.0.6,port=7005,name=172.23.0.6:7005,server_type=replica,redis_connection=Redis<ConnectionPool<Connection<host=172.23.0.6,port=7005,db=0>>>],
 [host=172.23.0.3,port=7002,name=172.23.0.3:7002,server_type=primary,redis_connection=Redis<ConnectionPool<Connection<host=172.23.0.3,port=7002,db=0>>>],
 [host=172.23.0.4,port=7003,name=172.23.0.4:7003,server_type=replica,redis_connection=Redis<ConnectionPool<Connection<host=172.23.0.4,port=7003,db=0>>>]]

In [6]:
len(items)

7249

In [7]:
item

{'id': 'AVpgMuGwLJeJML43KY_c',
 'prices.amountMax': 69.99,
 'prices.amountMin': 69.99,
 'prices.availability': 'Yes',
 'prices.condition': 'New',
 'prices.currency': 'USD',
 'prices.dateSeen': '2017-10-10T05:00:00Z',
 'prices.isSale': 'False',
 'prices.merchant': 'Bestbuy.com',
 'prices.shipping': 'nan',
 'prices.sourceURLs': 'https://www.bestbuy.com/site/boytone-2500w-2-1-ch-home-theater-system-black-diamond/4784804.p%25253FskuId%25253D4784804',
 'asins': 'B018K251JE,B00VILQKQ8',
 'brand': 'Boytone',
 'categories': 'Stereos,Portable Bluetooth Speakers,TV, Video & Home Audio,Speaker Systems,Portable Audio & Video,Electronics,See more Black BOYTONE Bt-210f 30 Watt FM Radio Bluetoo...,Speakers,Home Audio & Theater,All Home Speakers,Consumer Electronics,See more BOYTONE Bt-210f Bluetooth Wireless Speaker Mp3...,Home Theater Systems,MP3 Player Accessories,Home Audio,Audio,Cell,Stereo Shelf Systems',
 'dateAdded': '2015-05-18T14:14:56Z',
 'dateUpdated': '2018-06-13T19:39:02Z',
 'ean': 'nan'

In [8]:
for item in items:
    rc.set(item['id'], json.dumps(item))
    rc.hset(name='hset_' + item['id'], mapping=item)
    rc.lpush('devices', json.dumps(item))
    rc.zadd('sorted_set_devices', {item['id']: item['prices.amountMin']})